# Using Agent capability with Langchain

To create agent in langchain we need 3 things 
1. Docstring - What is this tool supposed to do. this should include what when and how this tool should be used for LLM to understand
2. Parameter Names - What these parameter names are e.g. send_parms(employee_id:int, employee_name:str) instead of send_params(params_1:int, params_2:str) 
3. If parameters names are not clear we should include the details of parameters in docstring 

In [1]:
from dotenv import load_dotenv 
load_dotenv(override=True)

True

In [2]:
import os
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [3]:
# set langsmit tracing off 
# os.environ['LANGSMITH_TRACING'] = "false"

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

# create llms 
google_llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-3-flash-preview")
openai_llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
ollama_llm = ChatOllama(temperature=0, model="llama3.1")

### Creating tools

In [5]:
from langchain_core.tools import tool 

@tool
def add(x:float, y:float, z:float|None=None) -> float: 
    "Add 'x' and 'y' "
    return x + y 

@tool
def substract(x:float, y:float) -> float: 
    "Substract 'y' from 'x' "
    return x - y 

@tool
def multiply(x:float, y:float) -> float: 
    "Multiply 'x' and 'y' "
    return x * y 

@tool
def divide(x:float, y:float) -> float: 
    "Divide 'x' by 'y' "
    return x / y 

@tool
def exponential(x:float, y:float) -> float: 
    "Raise 'x' to the power of 'y' "
    return x ** y

In [6]:
add

StructuredTool(name='add', description="Add 'x' and 'y'", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x10fa08e00>)

In [7]:
add.name

'add'

In [8]:
# see the arugument schema (pay attention to z which can be number of None and value is defaulted to None)
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y' ",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'},
  'z': {'anyOf': [{'type': 'number'}, {'type': 'null'}],
   'default': None,
   'title': 'Z'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

Reverting the tool add to include only x and y 

In [9]:
@tool
def add(x:float, y:float) -> float: 
    "Add 'x' and 'y' "
    return x + y 

### Creating Agent
```
Agent in LCEL is constructed as - 
agent = (
    <input parameters, including chat history and user query><br>
    | <prompt> <br>
    | <LLM with tools> <br>
)

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 

prompt = ChatPromptTemplate([
    ("system", (
        "You are a helpful assistant. When answering user's query you should first use one of the tool provided. "
        "After using the tool, the tool output will be provided in scratchpad below."
        "If you have the answer in scratchpad, you should not use anymore tools and instead answer directly to user."
    )),
    MessagesPlaceholder(variable_name="chat_history"), 
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

``` 
Agent scratchpad will have data in ToolMessage() format telling LLM that this is output from the tool

In [11]:
# provide reference of tools to LLM 
from langchain_core.runnables.base import RunnableSerializable
tools = [add, multiply, divide, substract, exponential]

agent:RunnableSerializable = (
    {
        "input": lambda x: x['input'], 
        "chat_history": lambda x : x['chat_history'], 
        "agent_scratchpad": lambda x : x.get("agent_scratchpad", [])
    }
    | prompt 
    | openai_llm.bind_tools(tools, tool_choice="any")
)

In [ ]:
# run this agent and AIMessage content will be empty as this is going to make a call to a tool
tool_call = agent.invoke({
    "input": "What is 5 + 10", 
    "chat_history": []
})
tool_call 

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 217, 'total_tokens': 234, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Csp8njxAG8jK1KGcDDX1FOjt8vMEC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7446-71dd-7dd0-b641-b8238f501e56-0', tool_calls=[{'name': 'add', 'args': {'x': 5, 'y': 10}, 'id': 'call_S0ensG8peZEfuLaBiekLHlVU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 217, 'output_tokens': 17, 'total_tokens': 234, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 5, 'y': 10},
  'id': 'call_S0ensG8peZEfuLaBiekLHlVU',
  'type': 'tool_call'}]

In [14]:
# now call the tool for that create a dictionary with name2tool 
name2tool = {tool.name : tool.func for tool in tools}
name2tool 

{'add': <function __main__.add(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'divide': <function __main__.divide(x: float, y: float) -> float>,
 'substract': <function __main__.substract(x: float, y: float) -> float>,
 'exponential': <function __main__.exponential(x: float, y: float) -> float>}

In [16]:
# now execute the tool
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]] (
    **tool_call.tool_calls[0]["args"]
)
tool_exec_content

15

``` 
Now we have the answer, shove this answer to agent scratchpad as Tool message for LLM to understand

In [17]:
from langchain_core.messages import ToolMessage
tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id = tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 5 + 10", 
    "chat_history": [], 
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 247, 'total_tokens': 264, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CspQGdgGVHsXZcNc8DNjVRRoNwwt5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7456-f7cc-7ae2-9c39-a15a385849ac-0', tool_calls=[{'name': 'add', 'args': {'x': 5, 'y': 10}, 'id': 'call_yuNpKJkHUDZWQy1LkZyr6cuY', 'type': 'tool_call'}], usage_metadata={'input_tokens': 247, 'output_tokens': 17, 'total_tokens': 264, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

``` 
Since the tool_choice is set to any content field of AI message will be empty. To get the output you can have 2 ways - 
a. set tool_choice="auto"; in this case final answer will appear in content field. 
b. set tool_choice="any/required"; and have final_output tool created which will have final answer. (Preferred way!)

In [18]:
@tool 
def final_answer(answer:str, tools_used:list[str]) -> str:
    """ Use this tool to provide the final answer to the user. 
    The answer should be in natural language as this will be provided to user directly.
    tools_used should contain list of tools used within the `scratchpad`
    """

    return {'answer': answer, "tools_used": tools_used}

In [28]:
# provide reference of tools to LLM 
from langchain_core.runnables.base import RunnableSerializable
tools = [final_answer, add, multiply, divide, substract, exponential]

agent:RunnableSerializable = (
    {
        "input": lambda x: x['input'], 
        "chat_history": lambda x : x['chat_history'], 
        "agent_scratchpad": lambda x : x.get("agent_scratchpad", [])
    }
    | prompt 
    | openai_llm.bind_tools(tools, tool_choice="any")
)

In [20]:
# run this agent and AIMessage content will be empty as this is going to make a call to a tool
tool_call = agent.invoke({
    "input": "What is 5 + 10", 
    "chat_history": []
})
tool_call 

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 279, 'total_tokens': 296, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CspcVz2yzK0Hx7WMyA1WfpiSrAlkR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7462-8d79-7850-9627-de857c0e1066-0', tool_calls=[{'name': 'add', 'args': {'x': 5, 'y': 10}, 'id': 'call_jsafMCjpnkWoHkjztDNRaMFL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 279, 'output_tokens': 17, 'total_tokens': 296, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [21]:
# now execute the tool
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]] (
    **tool_call.tool_calls[0]["args"]
)
tool_exec_content

15

In [22]:
from langchain_core.messages import ToolMessage
tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id = tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 5 + 10", 
    "chat_history": [], 
    "agent_scratchpad": [tool_call, tool_exec]
})

In [23]:
out

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 309, 'total_tokens': 336, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CspdNNOYQsvJ6yfKDfephj6eNHdem', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7463-60ca-7680-8481-a2ef25284762-0', tool_calls=[{'name': 'final_answer', 'args': {'answer': '5 + 10 equals 15.', 'tools_used': ['functions.add']}, 'id': 'call_77hOPBTbEs11gsndIklSoCxz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 309, 'output_tokens': 27, 'total_tokens': 336, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

In [24]:
# now execute the tool
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]] (
    **tool_call.tool_calls[0]["args"]
)
tool_exec_content

15

In [25]:
out.tool_calls

[{'name': 'final_answer',
  'args': {'answer': '5 + 10 equals 15.', 'tools_used': ['functions.add']},
  'id': 'call_77hOPBTbEs11gsndIklSoCxz',
  'type': 'tool_call'}]

``` 
Calling one after another manaully is not advisable. Instead create a class called CustomAgentExecutor 

In [32]:
# now call the tool for that create a dictionary with name2tool 
name2tool = {tool.name : tool.func for tool in tools}
name2tool 

{'final_answer': <function __main__.final_answer(answer: str, tools_used: list[str]) -> str>,
 'add': <function __main__.add(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'divide': <function __main__.divide(x: float, y: float) -> float>,
 'substract': <function __main__.substract(x: float, y: float) -> float>,
 'exponential': <function __main__.exponential(x: float, y: float) -> float>}

In [40]:
import json
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage

class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations:int=3):
        self.chat_history = [] 
        self.max_iterations = max_iterations
        self.agent:RunnableSerializable = (
            {
                "input": lambda x: x["input"], 
                "chat_history": lambda x: x["chat_history"], 
                "agent_scratchpad": lambda x : x["agent_scratchpad"]
            }
            | prompt 
            | openai_llm.bind_tools(tools, tool_choice="any")
        )

    def invoke(self, input:str) -> dict:
        """ runs agent iteratively until get final answer """
        count = 0 
        agent_scratchpad = [] 

        # iterate till max_iterations 
        while count < self.max_iterations:
            tool_call = self.agent.invoke({
                "input": input, 
                "chat_history": self.chat_history, 
                "agent_scratchpad": agent_scratchpad
            })

            agent_scratchpad.append(tool_call)

            # extract tool name (assumption is llm is calling one tool at a time)
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = name2tool[tool_name](**tool_args)

            # create tool message
            tool_exec = ToolMessage(
                content=f"{tool_out}", 
                tool_call_id=tool_call_id
            )

            # add tool message to scratchpad 
            agent_scratchpad.append(tool_exec)

            print(f"{count}: {tool_name}({tool_args})")
            count+=1 

            if tool_name == "final_answer":
                break 
        
        answer = tool_out["answer"]
        self.chat_history.extend([
            HumanMessage(content=input), 
            AIMessage(content=answer)
        ])

        return json.dumps(tool_out)


In [41]:
name2tool

{'final_answer': <function __main__.final_answer(answer: str, tools_used: list[str]) -> str>,
 'add': <function __main__.add(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'divide': <function __main__.divide(x: float, y: float) -> float>,
 'substract': <function __main__.substract(x: float, y: float) -> float>,
 'exponential': <function __main__.exponential(x: float, y: float) -> float>}

In [43]:
agent_executor = CustomAgentExecutor()
agent_executor.invoke(input="What is 6 + 18?")

0: add({'x': 6, 'y': 18})
1: final_answer({'answer': '6 + 18 equals 24.', 'tools_used': ['functions.add']})


'{"answer": "6 + 18 equals 24.", "tools_used": ["functions.add"]}'

# Not good for complicated tasks i.e. 6 + 18*5 as it assumes calls are going to be sequential